In [115]:
import numpy as np
import torch
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F
import os

In [116]:

import sys
import numpy as np
# add gensim source code fro this path
sys.path.insert(0, "../../../")
sys.path.insert(0, "../../gensim/")

# import modules & set up logging
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
                
                

## Import the training data from file

In [5]:
from smart_open import smart_open
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in smart_open(os.path.join(self.dirname, fname), 'r'):
                yield line.split()

In [6]:
# point to sentences to be learned file here
my_sentences_fn = '../..//ULL/data/Practical2/wa/english/'
my_sentences_fn = '../..//ULL/data/Practical2/wa/english_full/'

sentences = MySentences(my_sentences_fn) # a memory-friendly iterator
print(list(sentences))

[['2', '.'], ['2', '.'], ['oh', ',', 'oh', '!'], ['oh', ',', 'oh', '!'], ['oh', ',', 'oh', '!'], ['let', 'us', 'remember', ',', 'Mr.', 'Speaker', ',', 'that', 'these', 'segments', 'of', 'our', 'society', 'form', 'the', 'backbone', 'of', 'our', 'economy', '.'], ['hear', ',', 'hear', '!'], ['hear', ',', 'hear', '!'], ['Mr.', 'Speaker', ',', 'my', 'question', 'is', 'directed', 'to', 'the', 'Minister', 'of', 'Transport', '.'], ['both', 'have', 'many', 'years', 'experience', 'in', 'the', 'manufacture', 'and', 'distribution', 'of', 'forest', 'products', '.'], ['1', '.'], ['another', 'point', 'I', 'should', 'like', 'to', 'discuss', 'is', 'the', 'right', 'of', 'the', 'supplier', 'to', 'choose', 'his', 'customers', 'as', 'he', 'sees', 'fit', '.'], ['i', 'am', 'getting', 'sick', 'and', 'tired', 'of', 'ministers', 'who', 'seem', 'to', 'have', 'some', 'hang', '-', 'up', 'with', 'regard', 'to', 'the', 'collective', 'bargaining', 'process', '.'], ['there', 'is', 'just', 'one', 'specific', 'item', 'm

1. Clean stopwords
2. choose most frequent words replace 'the' etc with unknowns, also stopwords
3. 

# Pytorch  basic skipgram

In order to learn the embeddings we need a corpus and a method to read it and create the dictionary

In [9]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

1. Create context and targer pairs from the corpus

context: length k (k words surrounding the target) - it is the index of each context word in the vocabulary
eg. tensor([ 37,  30,   7,   6])


target: length 1 - index of target word
eg. tensor([40])

pytorch notes:
Note that the input to NLLLoss is a vector of log probabilities, and a target label. It doesn’t compute the log probabilities for us. This is why the last layer of our network is log softmax. 


In [12]:
mode="full"
if mode=='full':
    my_sentences_fn = '../..//ULL/data/Practical2/wa/english_full/'
else:
    my_sentences_fn = '../..//ULL/data/Practical2/wa/english/'

sentences = MySentences(my_sentences_fn) # a memory-friendly iterator


In [29]:
# Create the training data

vocabulary = []
for sentence in sentences:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}


vocabulary_size = len(vocabulary)

window_size = 2
context_size = window_size
idx_pairs = []
data=[]
bayesian_data=[]
# for each sentence
for sentence in sentences:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        contexts_for_word = []

        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            data.append((context_word_idx, indices[center_word_pos]))
            
            # this is used for bayesian to keep all contexts  of a word together
            contexts_for_word.append(context_word_idx)
        bayesian_data.append((contexts_for_word, indices[center_word_pos]))



vocab = vocabulary
vocab_size = vocabulary_size

In [17]:
print(data[10])

(3, 2)


In [ ]:
data = []
labels = []
for sentence in sentences:
    for i in range(len(sentence)):
        for j in range(i-window, i+window+1):
            if i != j and j>=0 and j<len(sentence):
                if (not (sentence[i] in stopWords or sentence[i] in string.punctuation or sentence[i].isdigit() or
                    sentence[j] in stopWords or  sentence[j] in string.punctuation or sentence[j].isdigit())):
                    # data.append(sentence[i])
                    # labels.append(sentence[j])
                    if sentence[i] in vocab:
                        data.append(w2i[sentence[i]])
                    else:
                        data.append(w2i[unk])
                    if sentence[j] in vocab:
                        labels.append(w2i[sentence[j]])
                    else:
                        labels.append(w2i[unk])

# Implementation 2 

In [18]:
  
class skipgram(nn.Module):  
    def __init__(self, vocab_size, embedding_dim):  
        super(skipgram,self).__init__()   
        self.embeddings = nn.Embedding(vocab_size, embedding_dim) # embeddings
        self.linear1 = nn.Linear(embedding_dim, vocab_size) # 

  
    def forward(self, cword_var): 
        embeds = self.embeddings(cword_var)  
        out = self.linear1(embeds)  
        log_probs = F.log_softmax(out)  
        return log_probs  
 

In [19]:
 
def make_context_vector(context, word_to_ix):  
    idxs = [word_to_ix[w] for w in context]  
    tensor = torch.LongTensor(idxs)  
    return Variable(tensor)  

In [20]:
CONTEXT_SIZE=context_size
  
embedding_dims = 50

  
#make_context_vector(data[0][0], word_to_ix)  # example  
  
# loss model optimizer  
losses = []  
loss_function = nn.NLLLoss()  
model = skipgram(vocab_size, embedding_dim=embedding_dims)  
optimizer = optim.SGD(model.parameters(), lr=0.001)  

epochs = 50
for epoch in range(epochs):  
    total_loss = torch.FloatTensor([0])  
    for context_words, cword in data: 

        context_var = torch.LongTensor([context_words]) 
        cword_var = Variable(torch.LongTensor([cword]))  
        model.zero_grad()  
        # input to model is the central word
        log_probs = model(cword_var)  
        loss = loss_function(log_probs, context_var)  
        loss.backward()  
        optimizer.step()  

        total_loss += loss.data  
    print("epoch", epoch, loss.data)
    #print(total_loss)
    losses.append(total_loss)  
print(losses) 

/Users/efiathieniti/miniconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


epoch 0 tensor(8.2481)
epoch 1 tensor(8.1486)
epoch 2 tensor(8.0160)
epoch 3 tensor(7.8684)
epoch 4 tensor(7.7211)
epoch 5 tensor(7.5762)
epoch 6 tensor(7.4337)
epoch 7 tensor(7.2947)
epoch 8 tensor(7.1604)
epoch 9 tensor(7.0318)
epoch 10 tensor(6.9099)
epoch 11 tensor(6.7955)
epoch 12 tensor(6.6890)
epoch 13 tensor(6.5908)
epoch 14 tensor(6.5010)
epoch 15 tensor(6.4195)
epoch 16 tensor(6.3461)
epoch 17 tensor(6.2805)
epoch 18 tensor(6.2222)
epoch 19 tensor(6.1708)
epoch 20 tensor(6.1255)
epoch 21 tensor(6.0860)
epoch 22 tensor(6.0515)
epoch 23 tensor(6.0216)
epoch 24 tensor(5.9958)
epoch 25 tensor(5.9736)
epoch 26 tensor(5.9546)
epoch 27 tensor(5.9384)
epoch 28 tensor(5.9248)
epoch 29 tensor(5.9133)
epoch 30 tensor(5.9037)
epoch 31 tensor(5.8958)
epoch 32 tensor(5.8894)
epoch 33 tensor(5.8842)
epoch 34 tensor(5.8802)
epoch 35 tensor(5.8772)
epoch 36 tensor(5.8751)
epoch 37 tensor(5.8736)
epoch 38 tensor(5.8728)
epoch 39 tensor(5.8726)
epoch 40 tensor(5.8728)
epoch 41 tensor(5.8734)
ep

## Save the embeddings

In [21]:
embeddings = model.embeddings.weight
import csv

embeddings_file="embeddings_final_" + str(mode) + str(epochs) + "_" +str(embedding_dims) + "_scratch.txt"
with open(embeddings_file, 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter = ' ')
    writer.writerow([vocabulary_size, embedding_dims])

    for i in range(embeddings.size()[0]):
        word = idx2word[i]

        embedding = embeddings[i, :]
        #print(embedding)
        embedding = list(embedding.data.numpy()) 
        line = [word] + embedding
        writer.writerow(line)

        

## Load the embeddings learned 

In [22]:
#embeddings_file="embeddings_full100_100_final.txt"

from gensim.models import KeyedVectors
#word_vectors = KeyedVectors.load_word2vec_format('../../../ULL/data/bow2.words.bz2', binary=False)  # C text format
word_vectors = KeyedVectors.load_word2vec_format(embeddings_file, binary=False)  # C binary format

In [23]:
print(word_vectors.similarity('message', 'issue'))


0.134572


## Calculate similarity

In [24]:
print(word_vectors.similarity('disturbed', 'pervasive'))


-0.0188806


In [25]:
embeddings_file

'embeddings_final_full100_50_scratch.txt'

In [26]:
word_vectors.evaluate_word_pairs("../../gensim/gensim/test/test_data/wordsim353.tsv")


((0.31042904413301226, 0.13097256206792052),
 SpearmanrResult(correlation=0.34923076923076918, pvalue=0.087060632485273348),
 92.91784702549575)

# Bayesian skip-gram

1. 
For each word/training point (xi) calculate elbo (loss)
xi - training points


### Create the word, contexts for bayesian

In [108]:
bayesian_data[200]


([119, 104, 25, 121], 120)

In [113]:
  
class bayesian_skipgram(nn.Module):  
    def __init__(self, vocab_size, embedding_dim):
        super(bayesian_skipgram,self).__init__()   
        
        hidden_dim = 150
        # embeddings lookup
        self.embeddings = nn.Embedding(vocab_size, embedding_dim) # embeddings
        self.linear1 = nn.Linear(embedding_dim, vocab_size) #
        
        
        # M linear projection [hid_dim, 2 * embd_dim]
        self.M_linear = nn.Linear(2*embedding_dim, embedding_dim)
        self.relu = nn.ReLU()
        
        self.U = nn.Linear(embedding_dim, embedding_dim) #
        self.W = nn.Linear(embedding_dim, embedding_dim) #



    def forward(self, cword_var, context_var): 
        
        # contexts_emb: Rcj, cword_emb: 
        contexts_emb = self.embeddings(context_var) 
        print("contexts_emb",contexts_emb.size())
        cword_emb = self.embeddings(cword_var).view(1,-1)
        cwords_emb = cword_emb.repeat(1,contexts_emb.size()[1],1)
        
        # concatenate word and context
        # (1, window_size, 2*emb_dim)
        word_contexts = torch.cat((cwords_emb, contexts_emb),2)
        print("word_contexts",word_contexts.size())
        
        #
        # linear projection, M
        # (1, window_size, 2*emb_dim)
        #
        word_contexts = self.M_linear(word_contexts)
        print("word_contexts_M",word_contexts.size())

        word_contexts = self.relu(word_contexts)
        
        # sum to collapse window_size dimension
        sum_contexts = torch.sum(word_contexts, 1)
        print("sum_contexts",sum_contexts.size())

        #
        # obtain the mu and sigma 
        #
        
        mu = self.U(sum_contexts)
        sigma = self.W(sum_contexts)
        print("mu",mu.size(),"sigma", sigma.size())
        
        out = self.linear1(sum_contexts)  
        log_probs = F.log_softmax(sum_contexts) 
        
        #
        # Do the sampling
        #
        
        return log_probs 

In [114]:
CONTEXT_SIZE=context_size
  
embedding_dims = 100

  
#make_context_vector(data[0][0], word_to_ix)  # example  
  
# loss model optimizer  
losses = []  
loss_function = nn.NLLLoss()  
model = bayesian_skipgram(vocab_size, embedding_dim=embedding_dims)  
optimizer = optim.SGD(model.parameters(), lr=0.001)  


epochs = 100
for epoch in range(epochs):  
    total_loss = torch.FloatTensor([0])  
    for context_words, cword in bayesian_data[100:]: 
        context_var = torch.LongTensor([context_words]) 
        cword_var = Variable(torch.LongTensor([cword]))  
        model.zero_grad()  
        # input to model is the central word
        log_probs = model(cword_var, context_var)  
        loss = loss_function(log_probs, context_var)  
        loss.backward()  
        optimizer.step()  

        total_loss += loss.data  
    print("epoch", epoch, loss.data)
    #print(total_loss)
    losses.append(total_loss)  
print(losses) 

contexts_emb torch.Size([1, 4, 100])
word_contexts torch.Size([1, 4, 200])
word_contexts_M torch.Size([1, 4, 100])
sum_contexts torch.Size([1, 100])
mu torch.Size([1, 100]) sigma torch.Size([1, 100])


/Users/efiathieniti/miniconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: multi-target not supported at /Users/soumith/minicondabuild3/conda-bld/pytorch_1524589329605/work/aten/src/THNN/generic/ClassNLLCriterion.c:22